# Data Preprocessing and Exploratory Data Analysis

## Data Preprocessing

In [ ]:
# importing the pandas library to work with the data
import pandas as pd

In [ ]:
# reading all flight data files from the csv files
data_cph = pd.read_csv("flight_data_cph.csv")
data_bll = pd.read_csv("flight_data_bll.csv")
data_aar = pd.read_csv("flight_data_aar.csv")

# merging all the dataframes into one
data = pd.concat([data_cph, data_bll, data_aar])

# sorting the data by the 'Departure Time' column
data = data.sort_values(by='Departure Time')
data

In [ ]:
# checking for missing values in the data 
print(data.isnull().sum())
# luckily, there are no missing values in the data

In [ ]:
# changing column names to make them more readable and easier to work with
data.rename(columns={
    'Search Date': 'departure_date',
    'Airline': 'airline',
    'Flight Number': 'flight_number',
    'Source City': 'source_city',
    'Departure Time': 'departure_time',
    'Number of Stops': 'number_of_stops',
    'Arrival Time': 'arrival_time',
    'Destination City': 'destination_city',
    'Duration (Minutes)': 'duration_minutes',
    'Price': 'price'
}, inplace=True)
# changing the names in the 'source_city' columns to their english names
data['source_city'] = data['source_city'].str.replace('København', 'Copenhagen')
data['source_city'] = data['source_city'].str.replace('Århus', 'Aarhus')
data

In [ ]:
# renaming the 'Price' column to 'price_in_DKK'
data = data.rename(columns = {'price': 'price_in_DKK'})

# removing the "kr." from the 'price_in_DKK' column in each row for easier analysis
data['price_in_DKK'] = data['price_in_DKK'].str.replace('kr.', '')

# removing the excess decimal point from the 'Flight Price in DKK' column in each row for easier analysis
data['price_in_DKK'] = data['price_in_DKK'].str.replace('.', '')

In [ ]:
# checking the data types of each column
print(data.dtypes)

In [ ]:
# converting the 'departure_date', 'departure_time', and 'arrival_time' columns to datetime because they are better represented as datetime objects as they are dates and times
data[['departure_date', 'departure_time', 'arrival_time']] = data[['departure_date', 'departure_time', 'arrival_time']].apply(pd.to_datetime)
# converting 'price_in_DKK' to float because it is a numerical value and should be represented as such
data['price_in_DKK'] = data['price_in_DKK'].astype(float)
# converting 'flight_number' to string because it is a categorical value and should be represented as such
data['flight_number'] = data['flight_number'].astype(str)
# converting 'number_of_stops' to string because it is a categorical value and should be represented as such
data['number_of_stops'] = data['number_of_stops'].astype(str)
# checking the data types of the columns after conversion
print(data.dtypes)

In [ ]:
# creating a new column based on which day of the week the flight is on
data['day_of_week'] = data['departure_date'].dt.day_name()

## EDA

In [ ]:
# importing the necessary libraries for data visualization
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for plotting

Statistical summary

In [ ]:
# getting the statistical summary of the price_in_DKK and duration_minutes columns
data[['price_in_DKK', 'duration_minutes']].describe().round(2)

In [ ]:
# checking the 90% quantile of the 'price_in_DKK' column
price_quantile = data['price_in_DKK'].quantile(0.90)
price_quantile

In [ ]:
# checking the 90% quantile of the 'duration_minutes' column
duration_quantile = data['duration_minutes'].quantile(0.90)
duration_quantile

In [ ]:
# removing the outliers from the 'price_in_DKK' column based on the 90% quantile
data = data[data['price_in_DKK'] <= price_quantile]
# removing the outliers from the 'duration_minutes' column based on the 90% quantile
data = data[data['duration_minutes'] <= duration_quantile]
# checking the statistical summary of the 'price_in_DKK' and 'duration_minutes' columns after removing the outliers
data[['price_in_DKK', 'duration_minutes']].describe().round(2)

Plot of the number of departures from each city

In [ ]:
# calculating the number of flights from each city
source_city_counts = data['source_city'].value_counts()

# plotting number of departures from each city
plt.figure(figsize=(8, 6))
plt.bar(source_city_counts.index, source_city_counts)
plt.title('Count Plot of City')
plt.xlabel('City')
plt.ylabel('Count')
plt.show()

Plot of the number of departures from each airline

In [ ]:
# calculating the number of flights from each airline
airline_counts = data['airline'].value_counts()

# plotting number of departures from each airline (inlcuding only the top 10 airlines)
plt.figure(figsize=(12, 6)) # changing the size of this plot for the airlines not to overlap
plt.bar(airline_counts.index[:10], airline_counts[:10])
plt.title('Count Plot of Airline')
plt.xlabel('Airline')
plt.xticks(rotation=45)
plt.ylabel('Count')
plt.show()

Plot of the number of departures on each weekday

In [ ]:
# calculating the number of flights from each day of the week
day_of_week_counts = data['day_of_week'].value_counts()

# plotting number of departures from each day of the week
plt.figure(figsize=(8, 6))
day_of_week_counts = day_of_week_counts.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.bar(day_of_week_counts.index, day_of_week_counts)
plt.title('Count Plot of Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Count')
plt.show()

Plotting the distribution of the number of stops

In [ ]:
# calculating the number of flights with different number of stops
number_of_stops_counts = data['number_of_stops'].value_counts()

# plotting number of departures with different number of stops in the order 0, 1, 2
plt.figure(figsize=(8, 6))
number_of_stops_counts = number_of_stops_counts.reindex(['0', '1', '2'])
plt.bar(number_of_stops_counts.index, number_of_stops_counts)
plt.title('Count Plot of Number of Stops')
plt.xlabel('Number of Stops')
plt.ylabel('Count')
plt.show()

Plotting the number of stops per airport

In [ ]:
# plotting the number of stops in accordance with the airpot of departure to see which airports have the most number of stops
plt.figure(figsize=(12, 6))
sns.countplot(data=data, x='source_city', hue='number_of_stops')
plt.title('Count Plot of Number of Stops by Source City')
plt.xlabel('Source City')
plt.ylabel('Count')
plt.show()

Boxplot for airline and price 

In [ ]:
# plotting the distribution of flight prices by source city
plt.figure(figsize=(12, 8)) # adjusting the figure size if needed
sns.boxplot(x='source_city', y='price_in_DKK', data=data)
plt.title('Boxplot of Price by Source City')
plt.xlabel('Source City')
plt.ylabel('Price in DKK')
plt.xticks(rotation=45) # rotating the x-axis labels for better readability
plt.show()

Line plot for weekdays and price

In [ ]:
# calculating the average price of flights by day of the week
average_prices = data.groupby('day_of_week')['price_in_DKK'].mean()

# setting the order of the weekdays and reindexing the data, while making sure that they are paired with the average price for the given day
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
average_prices = average_prices.reindex(weekday_order)

# plotting the average price of flights by day of the week
plt.figure(figsize=(10, 6)) # adjusting the figure size if needed
plt.plot(list(average_prices.index), average_prices.values, marker='o', linestyle='-')
plt.title('Average Price by Weekday')
plt.xlabel('Weekday')
plt.ylabel('Average Price in DKK')
plt.xticks(rotation=45) # rotating x-axis labels for better readability
plt.grid(True) # adding grid lines for better visualization
plt.show()

Scatter plot for departure time and price 

In [ ]:
# plotting the price trend by departure time
plt.figure(figsize=(12, 6)) # adjusting the figure size if needed
plt.scatter(data['departure_time'], data['price_in_DKK'])
plt.title('Scatter Plot of Price by Departure Time')
plt.xlabel('Departure Time')
plt.ylabel('Price in DKK')
plt.xticks(rotation=45) # rotating x-axis labels for better readability
plt.show()


Correlation matrix

In [ ]:
# selecting only the numerical columns
numerical_data = data.select_dtypes(include='number')

# plotting the correlation heatmap using only numerical data
plt.figure(figsize=(15, 10))
sns.heatmap(numerical_data.corr(), annot=True, fmt='.2f', cmap='Pastel2', linewidths=2)
plt.title('Correlation Heatmap (Numerical Data Only)')
plt.show()

## Feature Engineering

In [ ]:
# dropping the 'flight_number' column from the data
data = data.drop(columns=['flight_number'])
# dropping the 'destination_city' column from the data
data = data.drop(columns=['destination_city'])

In [ ]:
# lumping all the airlines with less than 25 flights into the 'Other' category
airline_counts = data['airline'].value_counts()
other_airlines = airline_counts[airline_counts < 25].index
data['airline'] = data['airline'].replace(other_airlines, 'Other')

In [ ]:
# creating a new column 'price_above_average' that indicates whether the price of the flight is above average
data['price_above_average'] = data['price_in_DKK'] > data['price_in_DKK'].mean()
# dropping the 'price_in_DKK' column from the data
data = data.drop(columns=['price_in_DKK'])

In [ ]:
# checking the distribution of the 'price_above_average' column in percentages in order to better be able to assess the accuracy of the model
price_above_average_distribution = data['price_above_average'].value_counts(normalize=True) * 100
price_above_average_distribution

In [ ]:
# creating a new column based on the number of days between the departure date and 2024-04-28
data['days_to_departure'] = (pd.to_datetime('2024-04-28') - data['departure_date']).dt.days
# multiplying the 'days_to_departure' column by -1 to make the values positive
data['days_to_departure'] = data['days_to_departure'] * -1
# dropping the 'departure_date' column as it is no longer needed
data = data.drop(columns=['departure_date'])

In [ ]:
# creating a column based on the time of the day the flight is on (morning, afternoon, evening, or night)
data['time_of_day'] = pd.cut(data['departure_time'].dt.hour, bins=[0, 6, 12, 18, 24], labels=['night', 'morning', 'afternoon', 'evening'])
# dropping the 'departure_time' and 'arrival_time' columns as they are no longer needed
data = data.drop(columns=['departure_time', 'arrival_time'])

In [ ]:
# one hot encoding the categorical columns
data = pd.get_dummies(data)
data

In [ ]:
# saving the cleaned data to a new csv file
data.to_csv('cleaned_flight_data.csv', index=False)